In [ ]:
# 해당 파일은 python 3.10.9에서 실행 되었습니다.

In [ ]:
# 시작할 때 사용할 패키지 파일을 전부 입력한다.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
from scipy import stats
from statannot import add_stat_annotation
# 그래프 한글 표시 요류 관련 해결 방법
plt.rc('font', family='Malgun Gothic')

In [ ]:
# 자료불러오기
df = pd.read_excel('미니프로젝트-쇼핑몰 실습데이터.xlsx') # 내부 데이터로 공유 불가
df.head(2)

# 할부기간 nan값 1개월로 변경

In [ ]:
# 할부기간 종류 확인
df['할부기간'].unique()

In [ ]:
# 할부기간 nsn값 1개월로 변경
df['할부기간'] = df['할부기간'].fillna('1개월')
df.head(2)

# 처리상태 종류 확인 및 구매확정 이외 삭제

In [ ]:
df['처리상태'].unique()

In [ ]:
df['처리상태'].value_counts()

In [ ]:
df1 = df[df['처리상태'] == '구매확정']
df1.head(2)

# 불필요 열 삭제
제조사, 초도상품, 제작문구 내역

In [ ]:
df1['제조사'].value_counts()

In [ ]:
# nan값 확인
df1['제조사'].isna().value_counts()

In [ ]:
df1['제작문구 내역'].unique()

In [ ]:
df1['초도상품'].unique()

In [ ]:
df2 = df1.drop(labels = ['제조사', '제작문구 내역', '초도상품'], axis = 1)
df2.head(2)

# 이외 nan값 제거

In [ ]:
df3 = df2.dropna()
df3.head(2)

# 결제 방법 간소화

In [ ]:
# 결제방법 확인
df3['결제방법'].unique()

In [ ]:
df4 = df3.replace(['신 + 포', '신 + 적 + 포', '신 + 적', '신 + 웰컴'],'신용카드')

In [ ]:
df5 = df4.replace('적 + 포','적립금')

In [ ]:
df6= df5.replace(['후 + 포', '후 + 적 + 포', '후 + 적'],'후불')

In [ ]:
df7 = df6.replace(['후 + 포', '후 + 적 + 포', '후 + 적'],'후불')

In [ ]:
df8 = df7.replace(['정 + 포', '정 + 적 + 포', '정 + 적'],'정기결제')

In [ ]:
df9 = df8.replace(['현 + 적', '현 + 포', '현 + 적 + 포'],'현금간편결제')

In [ ]:
df9['결제방법'].unique()

In [ ]:
# target 이외의 이상 값 항목 삭제
target = ['신용카드', '가상계좌', '포인트', '적립금', '웰컴마일', '후불', '정기결제', '현금간편결제']
df10 = df9[df9['결제방법'].apply(lambda x : x in target)]

In [ ]:
df10['결제방법'].unique()

In [ ]:
# 결제방법 '['신용카드', '가상계좌', '포인트', '적립금', '웰컴마일', '후불', '정기결제', '현금간편결제']

# 주문일자에서 주문 시간 제거

In [ ]:
# 주문일자 문자열로 변경
df10['주문일자'] = df10['주문일자'].astype('str')
df10.head(2)

In [ ]:
# '주문일자'에서 주문시간 제거
df10['주문일자'] = df10['주문일자'].apply(lambda x: x.split(' ')[0])

In [ ]:
# '주문일자'에서 '구매 연도'와 '구매 월' 열 추가
df10['구매 연도'] = df10['주문일자'].apply(lambda x: x.split('-')[0])
df10['구매 월'] = df10['주문일자'].apply(lambda x: x.split('-')[1])

In [ ]:
# '주문일자'에서 연도와 월만 가져와서'구매 연월' 열 추가
df10['구매 연월'] = df10['주문일자'].apply(lambda x: x.split('-')[0]) + '-' + df10['주문일자'].apply(lambda x: x.split('-')[1])
df10.head(2)

# 1. 입점 기업별 RFM 분석(3그룹) 진행

In [ ]:
# '업체명'과 '주문일자'로 데이터프레임 만들기
df11 = df10[['업체명','주문일자']].drop_duplicates()
df11

In [ ]:
# 마지막 주문일자 확인
df11['주문일자'].max()

In [ ]:
# 첫 주문일자 확인
df11['주문일자'].min()

In [ ]:
# '주문일자'에서 중복되는 항목 제외하고 해당 내용만 데이터프레임 만들기
df10[['주문일자']].drop_duplicates()

In [ ]:
# 업체별로 주문일수 
d11 = df11['업체명'].value_counts().reset_index()
d11

In [ ]:
# 업체별로 주문일수 통계
df11['업체명'].value_counts().reset_index().describe().astype('int64')

In [ ]:
# 1그룹
R1 = (d11['업체명'] >= 530).value_counts()
R1

In [ ]:
# 3그룹
R2 = (d11['업체명'] < 151).value_counts()
R2

In [ ]:
# 1,2,3그룹 전체 수
R3 = (d11['업체명'] >= 0).value_counts()
R3

In [ ]:
# 2그룹
print(R3 - R1 - R2)

In [ ]:
# 업체별 총 주문 횟수
df12 = df10['업체명'].value_counts().reset_index()
df12

In [ ]:
# 총 주문 횟수
df12['업체명'].sum()

In [ ]:
# 업체별로 주문건수 통계
df12['업체명'].describe().astype('int64')

In [ ]:
# 1그룹
F1 = (df12['업체명'] >= 1903).value_counts()
F1

In [ ]:
# 3그룹
F2 = (df12['업체명'] < 952).value_counts()
F2

In [ ]:
# 1,2,3그룹 전체 수
F3 = (df12['업체명'] >= 0).value_counts()
F3

In [ ]:
# 2그룹
print(F3 - F1 - F2)

In [ ]:
# 업체별 총 판매금액 
df13 = pd.DataFrame(df10.groupby('업체명')['판매금액'].sum().reset_index())
df13

In [ ]:
# 총 판매금액 
df13['판매금액'].sum()

In [ ]:
# 업체별로 총 판매금액 통계
df13['판매금액'].describe().astype('int64')

In [ ]:
# 1그룹
M1 = (df13['판매금액'] >= 86156557).value_counts()
M1

In [ ]:
# 3그룹
M2 = (df13['판매금액'] < 43078279).value_counts()
M2

In [ ]:
# 1,2,3그룹 전체 수
M3 = (df13['판매금액'] >= 0).value_counts()
M3

In [ ]:
# 2그룹
print(M3 - M1 - M2)

# 2. 매출 시각화
a. 월별, 연도별 매출

b. 월별 순수익 (처리 상황, 할부 기간 고려)

In [ ]:
# 구매 연월별 총판매금액
df10.groupby('구매 연월')['판매금액'].sum().reset_index()

In [ ]:
# barplot
sns.barplot(df10, x = '구매 연월', y = '판매금액')
plt.xticks(rotation = -90)

In [ ]:
# lineplot
sns.lineplot(df10, x = '구매 연월', y = '판매금액')
plt.xticks(rotation = -90)

In [ ]:
# pie
px.pie(df10, values = '판매금액', names = '구매 연월', title = '월별 판매 금액')

In [ ]:
# 구매 연도별 총판매금액
df10.groupby('구매 연도')['판매금액'].sum().reset_index()

In [ ]:
px.pie(df10, values = '판매금액', names = '구매 연도', title = '연도별 판매 금액')

# 3. 결제 방법에 따른 분석
a. 결제 방법과 결제금액의 연관성 분석

b. 결제방법은 맨 앞의 한가지만 사용한 것으로 간주

In [ ]:
# 결제방법 별 총 판매 금액
df20 = df10.groupby('결제방법')['판매금액'].sum().reset_index()
df20

In [ ]:
# barplot
sns.barplot(df20, x = '결제방법', y = '판매금액')

In [ ]:
# pie
px.pie(df20, values = '판매금액', names = '결제방법', title = '결제 방법별 판매 금액')

In [ ]:
# boxplot
sns.boxplot(df10, x = '결제방법', y = '판매금액')

In [ ]:
# 스피어만 등위상관계수
stats.spearmanr(df10['결제방법'],df10['판매금액'])

In [ ]:
# 켄달의 등위상관계수
stats.kendalltau(df10['결제방법'],df10['판매금액'])

# 1. 매출 시각화 2
a. 최대 매출 상품 3종류 집계

b. 주문 연도에 따른 해당 상품의 매출 증감 분석

In [ ]:
# 상품별 총 판매 금액
df21 = df10.groupby('상품명')['판매금액'].sum().reset_index()
df21

In [ ]:
# 상품별 총 판매 금액 상위 3 상품
df21.nlargest(3, '판매금액', keep='first')

In [ ]:
# 상품별 총 판매 금액 상위 3 상품만 따로 표기
target = ['[신세계] 신세계 상품권-4만원권', '[스마트 HBE] 학습 전용 태블릿-일반결제', '[스마트 HBE] 학습 전용 태블릿-정기결제']
df22 = df10[df10['상품명'].apply(lambda x : x in target)]
df22

In [ ]:
# 상품별 총 판매 금액 상위 3 상품 연도별 판매 금액
df23 = df22.groupby(['상품명', '구매 연도']).agg({'판매금액' : sum}).reset_index()
df23

In [ ]:
# 상품별 총 판매 금액 상위 3 상품 연도별 판매 금액 그래프
px.line(df23.sort_values(by = '구매 연도'), x = '구매 연도', y = '판매금액', color = '상품명', markers = True)

In [ ]:
# 상품별 총 판매 금액 상위 3 상품 월별 판매 금액
df24 = df22.groupby(['상품명', '구매 월']).agg({'판매금액' : sum}).reset_index()
df24

In [ ]:
# 상품별 총 판매 금액 상위 3 상품 월별 판매 금액 그래프
px.line(df24, x = '구매 월', y = '판매금액', color = '상품명', markers = True)

# 2. 연관성 분석
a. 주문한 달과 판매금액의 상관관계 분석

In [ ]:
# 스피어만 상관계수
stats.spearmanr(df10['구매 월'],df10['판매금액'])

In [ ]:
# 켄달의 타우
stats.kendalltau(df10['구매 월'],df10['판매금액'])

In [ ]:
# barplot
sns.barplot(df10, x = '구매 월', y = '판매금액')